In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
import csv
import geopy.distance

### Primer acercamiento a Machine Learning (KNN)

In [2]:
dfTest = pd.read_csv('../data/TRAIN_TEST_corrected/test_corrected.csv')
dfTrain = pd.read_csv('../data/TRAIN_TEST_corrected/train_corrected.csv')
dfExternalCoords = pd.read_csv('../data/externalData/coordenadas_barrios.csv')

In [3]:
dfTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 11 columns):
id                     14166 non-null int64
year_created           14166 non-null int64
month_created          14166 non-null int64
day_created            14166 non-null int64
property_type          14166 non-null object
place_name             14166 non-null object
state_name             14166 non-null object
lat                    10487 non-null float64
lon                    10487 non-null float64
surface_total_in_m2    11853 non-null float64
rooms                  7501 non-null float64
dtypes: float64(4), int64(4), object(3)
memory usage: 1.2+ MB


In [4]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004797 entries, 0 to 1004796
Data columns (total 12 columns):
id                     1004797 non-null object
year_created           1004797 non-null int64
month_created          1004797 non-null int64
day_created            1004797 non-null int64
property_type          1004797 non-null object
place_name             1004797 non-null object
state_name             1004797 non-null object
lat                    743693 non-null float64
lon                    743682 non-null float64
surface_total_in_m2    1004797 non-null float64
rooms                  594683 non-null float64
price_aprox_usd        1004797 non-null float64
dtypes: float64(5), int64(3), object(4)
memory usage: 92.0+ MB


In [5]:
dfExternalCoords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 4 columns):
place                        438 non-null object
lat                          435 non-null float64
lon                          435 non-null float64
distance_to_obelisco_mtrs    435 non-null float64
dtypes: float64(3), object(1)
memory usage: 13.8+ KB


In [6]:
# Filtro los 3 registros de los cuales no tengo informacion de location
dfExternalCoords = dfExternalCoords.loc[pd.notnull(dfExternalCoords.lat)]
dfExternalCoords.head()

,place,lat,lon,distance_to_obelisco_mtrs
0,Country Club Las lajas,-34.590190,-58.933716,50672.525719
1,Barrio Cerrado El Lucero,-34.422229,-58.763899,40471.776026
2,Caballito,-34.620928,-58.445874,6198.266937
3,Gregorio de Laferrere,-34.749748,-58.584591,24668.602907
4,Acassuso,-34.477361,-58.499270,17699.585005


## ToDo:
+ Resolver NaN's en lat / lon con externalData.
+ Reemplazar place_name con distancia al obelisco.
+ Z-codes Standardization.
+ OneHotEncode property_type y state_name.
+ Make Submit & Comparison
+ Cross Validation K-Folds -> Make Submit & Comparison

In [7]:
dfTrain.head(2)

,id,year_created,month_created,day_created,property_type,place_name,state_name,lat,lon,surface_total_in_m2,rooms,price_aprox_usd
0,1771e4252915a911b8eb4964cde2790d5623bd33,2013,8,28,apartment,Caseros,Bs.As. G.B.A. Zona Oeste,-34.600116,-58.565334,62.0,3.0,130650.48
1,d63560d1f8f797a6d412814a27ec570beafb9d90,2013,8,28,apartment,Villa Crespo,Capital Federal,-34.597274,-58.439479,44.0,2.0,78500.00


In [8]:
dfTest.head(2)

,id,year_created,month_created,day_created,property_type,place_name,state_name,lat,lon,surface_total_in_m2,rooms
0,3632,2017,8,24,apartment,Puerto Madero,Capital Federal,-34.610988,-58.363464,0.0,NaN
1,3633,2017,8,25,apartment,Palermo,Capital Federal,NaN,NaN,0.0,2.0


___
### Corriendo el primer KNN

In [9]:
dfUtil = dfTrain.loc[:, ('place_name', 'year_created')]\
                .groupby(by='place_name')\
                .count().reset_index()\
                .sort_values(by='year_created', ascending=False)\
                .rename(columns={'year_created':'place_ocurrences'})
print 'Cantidad de barrios con menos de 100 publicaciones:', dfUtil.loc[dfUtil.place_ocurrences < 100, ('place_ocurrences')].sum()

Cantidad de barrios con menos de 100 publicaciones: 7957


In [10]:
# ONE-TO-K ENCODING: PROPERTY_TYPE
prop_type_labelEncoder = preprocessing.LabelEncoder()
prop_type_labelEncoder.fit(np.array(list(dfTrain.property_type.unique()), dtype='object'))

trainPropTypeIntEncoded = prop_type_labelEncoder.transform(dfTrain.property_type)
testPropTypeIntEncoded = prop_type_labelEncoder.transform(dfTest.property_type)

enc = preprocessing.OneHotEncoder(sparse=False)
matrixTrainPropTypeEnc = enc.fit_transform(trainPropTypeIntEncoded.reshape(len(trainPropTypeIntEncoded), 1))
matrixTestPropTypeEnc = enc.transform(testPropTypeIntEncoded.reshape(len(testPropTypeIntEncoded), 1))

oneHotCodeCategoriesTrain = pd.DataFrame(matrixTrainPropTypeEnc, columns=prop_type_labelEncoder.classes_)
oneHotCodeCategoriesTest = pd.DataFrame(matrixTestPropTypeEnc, columns=prop_type_labelEncoder.classes_)

dfTrain = pd.concat([dfTrain.reset_index(drop=True), oneHotCodeCategoriesTrain.reset_index(drop=True)], 
                     axis=1).drop('property_type', axis=1)

dfTest = pd.concat([dfTest.reset_index(drop=True), oneHotCodeCategoriesTest.reset_index(drop=True)],
                    axis=1).drop('property_type', axis=1)

In [11]:
dfTrain.head(2)

,id,year_created,month_created,day_created,place_name,state_name,lat,lon,surface_total_in_m2,rooms,price_aprox_usd,PH,apartment,house,store
0,1771e4252915a911b8eb4964cde2790d5623bd33,2013,8,28,Caseros,Bs.As. G.B.A. Zona Oeste,-34.600116,-58.565334,62.0,3.0,130650.48,0.0,1.0,0.0,0.0
1,d63560d1f8f797a6d412814a27ec570beafb9d90,2013,8,28,Villa Crespo,Capital Federal,-34.597274,-58.439479,44.0,2.0,78500.00,0.0,1.0,0.0,0.0


In [12]:
dfTest.head(2)

,id,year_created,month_created,day_created,place_name,state_name,lat,lon,surface_total_in_m2,rooms,PH,apartment,house,store
0,3632,2017,8,24,Puerto Madero,Capital Federal,-34.610988,-58.363464,0.0,NaN,0.0,1.0,0.0,0.0
1,3633,2017,8,25,Palermo,Capital Federal,NaN,NaN,0.0,2.0,0.0,1.0,0.0,0.0


### Reemplazo de place_name y state_name por distancia al obelisco
> + Si tiene lat y lon la dejo y calculo su distancia al obelisco.
> + Si no tiene lat y lon, las estraigo de los datos externos traidos de Gmaps

In [13]:
# Calculo distancias al obelisco en donde pueda y en las otras dejo NaN :
def distanceToObeliscoMeters(seriesLat, seriesLon) :
    distance_2_obelisco = []
    obelisco = (-34.6037389, -58.3815704)
    
    for lat, lon in zip(seriesLat, seriesLon) :        
        dist_m = np.nan if (pd.isnull(lat) or pd.isnull(lon)) else geopy.distance.vincenty(obelisco, (lat, lon)).m
        distance_2_obelisco.append(dist_m)
        
    return pd.Series(distance_2_obelisco)

In [14]:
distancesObeliscoTrain = distanceToObeliscoMeters(dfTrain.lat, dfTrain.lon)
distancesObeliscoTest = distanceToObeliscoMeters(dfTest.lat, dfTest.lon)
dfTrain.insert(column='distance_to_obelisco_mtrs', loc=4, value=distancesObeliscoTrain)

In [15]:
dfUpdate = dfTrain.loc[pd.isnull(dfTrain.distance_to_obelisco_mtrs)].rename(columns={'place_name':'place'})\
                                                                    .merge(dfExternalCoords, how='inner', on=['place'])\
                                                                    .drop(['lat_x', 'lon_x', 'distance_to_obelisco_mtrs_x'], axis=1)\
                                                                    .rename(columns={'lat_y':'lat', 'lon_y':'lon', 'distance_to_obelisco_mtrs_y':'distance_to_obelisco_mtrs'})

In [16]:
dfTrain.set_index('id', drop=True, inplace=True)
dfUpdate.set_index('id', drop=True, inplace=True)
dfTrain.update(dfUpdate)

In [17]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1004797 entries, 1771e4252915a911b8eb4964cde2790d5623bd33 to eb2e1fe4ad4e7c326d5fb92602c7cac11bb856c6
Data columns (total 15 columns):
year_created                 1004797 non-null float64
month_created                1004797 non-null float64
day_created                  1004797 non-null float64
distance_to_obelisco_mtrs    1004777 non-null float64
place_name                   1004797 non-null object
state_name                   1004797 non-null object
lat                          1004777 non-null float64
lon                          1004777 non-null float64
surface_total_in_m2          1004797 non-null float64
rooms                        594683 non-null float64
price_aprox_usd              1004797 non-null float64
PH                           1004797 non-null float64
apartment                    1004797 non-null float64
house                        1004797 non-null float64
store                        1004797 non-null float64
dtypes: float

In [ ]:
# Lleno los valores faltantes (NaN's) con la clase predominante en esa columna

imp = preprocessing.Imputer(missing_values='NaN', strategy='most_frequent')
imp.fit(dfTrain[['rooms']])
dfTrain.rooms = imp.transform(dfTrain[['rooms']]).ravel()

imp = preprocessing.Imputer(missing_values='NaN', strategy='mean')
imp.fit(dfTrain[['lat']])
dfTrain.lat = imp.transform(dfTrain[['lat']]).ravel()
imp = preprocessing.Imputer(missing_values='NaN', strategy='mean')
imp.fit(dfTrain[['lon']])
dfTrain.lon = imp.transform(dfTrain[['lon']]).ravel()

imp = preprocessing.Imputer(missing_values='NaN', strategy='most_frequent')
imp.fit(dfTest[['rooms']])
dfTest.rooms = imp.transform(dfTest[['rooms']]).ravel()

imp = preprocessing.Imputer(missing_values='NaN', strategy='mean')
imp.fit(dfTest[['lat']])
dfTest.lat = imp.transform(dfTest[['lat']]).ravel()
imp = preprocessing.Imputer(missing_values='NaN', strategy='mean')
imp.fit(dfTest[['lon']])
dfTest.lon = imp.transform(dfTest[['lon']]).ravel()

imp = preprocessing.Imputer(missing_values='NaN', strategy='mean')
imp.fit(dfTest[['surface_total_in_m2']])
dfTest.surface_total_in_m2 = imp.transform(dfTest[['surface_total_in_m2']]).ravel()

In [ ]:
dfTrain.info()

In [ ]:
dfTest.info()

In [ ]:
# Genero TRAIN y TARGET
target = dfTrain.price_usd
train = dfTrain.drop('price_usd', axis=1)

testIds = dfTest.loc[:, ('id')]
testVals = dfTest.drop('id', axis=1)

In [ ]:
knn = KNeighborsRegressor(n_neighbors=30, weights='distance', 
                          algorithm='kd_tree', leaf_size=30, 
                          metric='euclidean', n_jobs=-1)

knn.fit(train, target)

In [ ]:
predictions = knn.predict(testVals)

In [ ]:
# Escribo a archivo

with open('../data/predictions/2nd_acercamiento.csv', 'w') as outfile:
        rows = [["id", "price_usd"]]

        for idTest, prediction in zip(testIds, predictions):
            rows.append([idTest, prediction])

        out_csv = csv.writer(outfile)
        out_csv.writerows(rows)